In [21]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
from tqdm import tqdm
from tenacity import retry, stop_after_attempt, wait_random_exponential
import os

import openai

from utils.embedding_utils import get_embedding, sliding_window

In [133]:
df = pd.read_csv("../data/db/df_text.csv")
df.head()

,source_type,text
0,paper,\begin{document}\n\n\preprint{MIT-CTP 5315}\n\...
1,paper,% ============================================...
2,paper,\begin{document}\n\n%\preprint{}\n\n\title{Pre...
3,paper,Once detector data has been processed and high...
4,paper,\begin{document}\n\preprint{MIT--CTP 5223}\n\n...


In [134]:
df_filtered = df

In [10]:
window_size = 256
stride = 192

text_chunks = []
embeddings = []

for i in tqdm(range(len(df_filtered))):
    text = df_filtered['text'].values[i].replace('\n', ' ').strip()
    text_chunks_i = list(sliding_window(text, window_size, stride))
    if len(text_chunks_i) > 80:
        text_chunks_i = text_chunks_i[:80]
    embeddings_i = [get_embedding(text) for text in text_chunks_i]
    text_chunks += text_chunks_i
    embeddings += embeddings_i

100%|████████████████████████████████████████████████████████| 65/65 [12:07<00:00, 11.20s/it]


In [11]:
embeddings = np.array(embeddings, dtype=np.float64)

In [64]:
data = [text_chunks]

# Transpose the data to have the inner lists as rows
transposed_data = list(map(list, zip(*data)))

# Column names for the DataFrame
columns = ['text_chunks']

# Create the DataFrame
df = pd.DataFrame(transposed_data, columns=columns)

In [65]:
df.to_csv('../data/db/text_chunks.csv', index=False)
np.save('../data/db/embeddings.npy', embeddings)

In [130]:
from sklearn.metrics.pairwise import cosine_similarity
from utils.embedding_utils import get_embedding

def semantic_search(query_embedding, embeddings):
    """Load context prompt."""
    similarities = cosine_similarity([query_embedding], embeddings)[0]
    ranked_indices = np.argsort(-similarities)
    return ranked_indices

In [195]:
emb = get_embedding("have you collaborated with rikab gambhir?")
semantic_search(emb, embeddings)

array([2751, 2737, 2705, ...,  110, 1660, 1651])

In [196]:
text_chunks[semantic_search(emb, embeddings)[0]]

'Fall 2019 Before MIT Teaching Fellow, Widely Applied Physics (Nima Arkani-Hamed), Harvard University, Fall 2005 Teaching Assistant, Linear Algebra (Thomas Banchoff), Brown University, Spring 2001 Math/Physics Tutor, Brown University, 1999 - 2000 Classroom Assistant, Algebra I and II, Textron/Chamber of Commerce Charter High School, Providence, RI, Fall 1999 Publications & Preprints Publications by Topic Inspire arXiv ORCID Google Scholar 2023 SHAPER: Can You Hear the Shape of a Jet? . Demba Ba, Akshunna S. Dogra, Rikab Gambhir, Abiy Tasissa, and Jesse Thaler. arXiv:2302.12266 . EPiC-GAN: Equivariant Point Cloud Generation for Particle Jets . Erik Buhmann, Gregor Kasieczka, and Jesse Thaler. arXiv:2301.08128 . 2022 Comparing Point Cloud Strategies for Collider Event Classification . Peter Onyisi, Delon Shen, and Jesse Thaler. arXiv:2212.10659 . Degeneracy Engineering for Classical and Quantum Annealing: A Case Study of Sparse Linear Regression in Collider Physics . Eric R. Anschuetz, L

In [197]:
from chatjesset import load_csv_file_from_gcs

csv_reader = load_csv_file_from_gcs("chatjesset.appspot.com", "text_chunks.csv")

ImportError: cannot import name 'storage' from 'google.cloud' (unknown location)